# The Model Using the food-101 Dataset to Classify Images of Food

We will first preprocess the data

In [32]:
import tensorflow_datasets as tfds

list_datasets = tfds.list_builders()
print('food101' in list_datasets)

True


In [33]:
(train_data,test_data),df_info = tfds.load('food101',
                                           split=['train','validation'], # in our case we have training and validation but some dataset have testing too
                                           shuffle_files=True, # we are shuffle our files to some randomness in our data
                                           as_supervised=True, # beacuse our dataset is supervised
                                           with_info=True) # getting metadata about our dataset

In [34]:
# Features of the dataset
df_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

### Reading all the names of the 101 different types of foods we are training on

In [35]:
# Getting the class names

class_names = df_info.features['label'].names
class_names



['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheesecake',
 'cheese_plate',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

### Preprocess the image data

In [38]:
import tensorflow as tf


def preprocess_image_data(images,labels,img_size=(224,224)):
  '''
  This function will reduce image size and change dtype int float32

  '''
  image  = tf.image.resize(images,img_size)

  return tf.cast(image,dtype=tf.float32),labels

In [40]:
# we are going to use map method to map our preprocess_image_data function for preprocessing

train_data = train_data.map(map_func=preprocess_image_data,num_parallel_calls=tf.data.AUTOTUNE)

# making into batches for training data

train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# preprocess and making batches for testing data

test_data  = test_data.map(map_func=preprocess_image_data,num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [41]:
train_data,test_data

(<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>,
 <PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>)

In [42]:
import os
import datetime
# creating the tensoboard callback
def create_tensorboard_callback(dir_name,experiment):
  date_time = datetime.datetime.now().strftime('%Y/%m/%d:%H-%M-%S')
  path = os.path.join(dir_name,experiment,date_time)
  return tf.keras.callbacks.TensorBoard(log_dir=path)

In [43]:
# model check point callback

checkpoint_path = 'model_checkpoints/cp.cpkt'

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         monitor='val_accuracy',
                                                         save_best_only=True,
                                                         save_weights_only=True,
                                                         verbose=0)

In [44]:
# start by importing it 
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16') # this will do the job

# Modeling

Now, we will import the necessary packages and then work on training the model

In [45]:
mixed_precision.global_policy()

<Policy "mixed_float16">

### Here is where we actually write the code for the convolutional neural network

In [46]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow as tf

# we are going t use the transfer learning - feature extraction

input_shape = (224,224,3)

# our base model will be EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet')
base_model.trainable = False

input = layers.Input(shape=input_shape,name='input_layer')

# data_augmentation = tf.keras.Sequential([ # this is for  data augmentation
#     preprocessing.RandomRotation(0.1),
#     preprocessing.RandomZoom(0.2),
#     preprocessing.RandomFlip(mode='horizontal'),
#     preprocessing.Rescaling(1/255.)
# ], name='data_augmentation')

x = base_model(input)
x = layers.GlobalAveragePooling2D(name='globalAvgPooling_layer')(x)

# x = layers.Dense(512, activation='relu')(x)
# x = layers.Dropout(0.2)(x)

x = layers.Dense(101,name='output_layer')(x)

output = layers.Activation(activation='softmax',dtype=tf.float32,name='prediction')(x)

model = tf.keras.Model(input,output,name='food_vision_model')

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

### Here, we train the CNN

In [49]:
feature_extraction_history = model.fit(train_data,
                                       epochs=5,
                                       steps_per_epoch=len(train_data),
                                       validation_data=test_data,
                                       validation_steps=int(.15*len(test_data)),
                                       callbacks=[checkpoint_callback])

Epoch 1/5
  86/2368 [>.............................] - ETA: 27:58 - loss: 3.9574 - accuracy: 0.1602

KeyboardInterrupt: 

## Evalaute the CNN on the test data

In [ ]:
fx_results = model.evaluate(test_data)
fx_results

790/790 [==============================] - 20s 25ms/step - loss: 1.0592 - accuracy: 0.7122


[1.0591827630996704, 0.7122375965118408]

In [ ]:
# lets make it up

base_model.trainable = True

for layer in base_model.layers[:-20]:
  layer.trainable = False

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4), # we are reducing the learning rate so pretrained weights does not change too much
              metrics=['accuracy'])

In [ ]:
for layer in model.layers:
  print(layer.name,layer.trainable)

input_layer True
efficientnetb0 True
globalAvgPooling_layer True
output_layer True
prediction True


In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=4) # if val loss decreases for 3 epochs in a row, stop training

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

In [ ]:
# creating the callbacks to reduce learning rate of the model
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=4,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [ ]:
best_model = model.fit(train_data,
                       epochs=100,
                       steps_per_epoch=len(train_data),
                       validation_data=test_data,
                       validation_steps=int(.15 * len(test_data)),
                       callbacks=[early_stopping,reduce_lr,model_checkpoint])

Epoch 1/100
2368/2368 [==============================] - 93s 35ms/step - loss: 1.1464 - accuracy: 0.6975 - val_loss: 0.9525 - val_accuracy: 0.7421
INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets
c:\Users\Luqman\Summer-2023-ML-Project\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
2368/2368 [==============================] - 82s 34ms/step - loss: 0.8822 - accuracy: 0.7642 - val_loss: 0.8960 - val_accuracy: 0.7511
INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets
c:\Users\Luqman\Summer-2023-ML-Project\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 3/100
2368/2368 [==============================] - 79s 33ms/step - loss: 0.7617 - accuracy: 0.7946 - val_loss: 0.8694 - val_accuracy: 0.7651
INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets


...

Epoch 4/100
2368/2368 [==============================] - 80s 33ms/step - loss: 0.6648 - accuracy: 0.8220 - val_loss: 0.8573 - val_accuracy: 0.7646
INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets
c:\Users\Luqman\Summer-2023-ML-Project\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 5/100
2368/2368 [==============================] - 81s 34ms/step - loss: 0.5773 - accuracy: 0.8460 - val_loss: 0.8561 - val_accuracy: 0.7656
INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets
c:\Users\Luqman\Summer-2023-ML-Project\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 6/100
2368/2368 [==============================] - 82s 34ms/step - loss: 0.5118 - accuracy: 0.8642 - val_loss: 0.8638 - val_accuracy: 0.7619
Epoch 7/100
2368/2368 [==============================] - 80s 34ms/step - loss: 0.4467 - accuracy: 0.8835 - val_loss: 0.8651 - val_accuracy: 0.7677
Epoch 8/100
2368/2368 [==============================] - 82s 34ms/step - loss: 0.3972 - accuracy: 0.8980 - val_loss: 0.8690 - val_accuracy: 0.7704
Epoch 9/100
2368/2368 [==============================] - 82s 34ms/step - loss: 0.3509 - accuracy: 0.9109 - val_loss: 0.8867 - val_accuracy: 0.7707

Epoch 00009: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.


In [ ]:
model.save(r'C:\Users\Luqman\Summer-2023-ML-Project\model')

INFO:tensorflow:Assets written to: C:\Users\Luqman\Summer-2023-ML-Project\model\assets


INFO:tensorflow:Assets written to: C:\Users\Luqman\Summer-2023-ML-Project\model\assets
c:\Users\Luqman\Summer-2023-ML-Project\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# GPU Check


In [1]:
import tensorflow as tf


# Check if GPU is available
print("Is a GPU available: ", tf.config.list_physical_devices('GPU'))

# Check which device GPU is using
print("GPU device name: ", tf.test.gpu_device_name())

tf.config.list_physical_devices('GPU')



Is a GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU device name:  /device:GPU:0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]